## Imports

In [1]:
from pymongo import MongoClient
from pprint import pprint

## Connect to mongo database

In [2]:
# Connect to Mongo server
host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Connect to mongo db
db = client.denormalised

In [3]:
db.list_collection_names()

['Application_Countries',
 'Warehouse_Colors',
 'Sales_OrderLines',
 'Warehouse_StockItemTransactions',
 'Application_StateProvinces',
 'Sales_Invoices',
 'Sales_Orders',
 'Purchasing_PurchaseOrders',
 'Application_DeliveryMethods',
 'Application_Cities',
 'Application_TransactionTypes',
 'Warehouse_StockItems',
 'Sales_InvoiceLines',
 'Warehouse_StockItemStockGroups',
 'Purchasing_PurchaseOrderLines',
 'Purchasing_SupplierCategories',
 'Application_PaymentMethods',
 'Sales_Customers',
 'Sales_CustomerCategories',
 'Purchasing_SupplierTransactions',
 'Application_People',
 'Warehouse_PackageTypes',
 'Purchasing_Suppliers',
 'Warehouse_StockGroups',
 'Sales_CustomerTransactions']

### Design Requirements

The CFO:  
- Would like to know the monthly order count?

In [133]:
# According to this link: https://docs.mongodb.com/v5.0/core/aggregation-pipeline/#:~:text=Pipeline%20Operators%20and%20Indexes,
# this query is not possible to be done with an index, because of the accumulator

query_1 = {
    '$group': {
        '_id' : {'$substr': ['$OrderDate', 0, 7]},
        'ordercount': {'$sum': 1}
    }
}

query_2 = {
    '$sort': {'_id': 1}
}

query_3 = {
    '$project': {
        '_id' : 0,
        'Month': '$_id',
        'ordercount' : 1
    }
}

pipeline = [query_1, query_2, query_3]

r = db.Sales_Orders.aggregate(pipeline)

result = list(r)

result

[{'ordercount': 1674, 'Month': '2013-01'},
 {'ordercount': 1139, 'Month': '2013-02'},
 {'ordercount': 1683, 'Month': '2013-03'},
 {'ordercount': 1696, 'Month': '2013-04'},
 {'ordercount': 1808, 'Month': '2013-05'},
 {'ordercount': 1675, 'Month': '2013-06'},
 {'ordercount': 1886, 'Month': '2013-07'},
 {'ordercount': 1537, 'Month': '2013-08'},
 {'ordercount': 1617, 'Month': '2013-09'},
 {'ordercount': 1618, 'Month': '2013-10'},
 {'ordercount': 1552, 'Month': '2013-11'},
 {'ordercount': 1565, 'Month': '2013-12'},
 {'ordercount': 1791, 'Month': '2014-01'},
 {'ordercount': 1538, 'Month': '2014-02'},
 {'ordercount': 1586, 'Month': '2014-03'},
 {'ordercount': 1739, 'Month': '2014-04'},
 {'ordercount': 1908, 'Month': '2014-05'},
 {'ordercount': 1887, 'Month': '2014-06'},
 {'ordercount': 2008, 'Month': '2014-07'},
 {'ordercount': 1642, 'Month': '2014-08'},
 {'ordercount': 1637, 'Month': '2014-09'},
 {'ordercount': 1921, 'Month': '2014-10'},
 {'ordercount': 1628, 'Month': '2014-11'},
 {'ordercou

- Would like to know the average monthly sales prices? 

In [139]:
query_1 = {
    '$unwind' : "$OrderLines"
}

query_2 = {
    '$group': {
        '_id' : {'$substr': ['$OrderDate', 0, 7]},
        'average_monthly_sales_prices': { '$avg': { '$multiply': [ "$OrderLines.Quantity", "$OrderLines.UnitPrice" ] } }
    }
}

query_3 = {
    '$project': {
        '_id' : 0,
        'Month': '$_id',
        'average_monthly_sales_prices' : 1
    }
}

query_4 = {
    '$sort': {'Month': 1}
}

pipeline = [query_1, query_2, query_3, query_4]

r = db.Sales_Orders.aggregate(pipeline)

result = list(r)

result

[{'average_monthly_sales_prices': 724.2648835447832, 'Month': '2013-01'},
 {'average_monthly_sales_prices': 757.187922705314, 'Month': '2013-02'},
 {'average_monthly_sales_prices': 735.9580812766748, 'Month': '2013-03'},
 {'average_monthly_sales_prices': 782.0304949190817, 'Month': '2013-04'},
 {'average_monthly_sales_prices': 800.637015265836, 'Month': '2013-05'},
 {'average_monthly_sales_prices': 777.4632071937056, 'Month': '2013-06'},
 {'average_monthly_sales_prices': 763.694343622795, 'Month': '2013-07'},
 {'average_monthly_sales_prices': 747.2962440340319, 'Month': '2013-08'},
 {'average_monthly_sales_prices': 767.9943616395371, 'Month': '2013-09'},
 {'average_monthly_sales_prices': 750.3137594275769, 'Month': '2013-10'},
 {'average_monthly_sales_prices': 770.5888844058907, 'Month': '2013-11'},
 {'average_monthly_sales_prices': 750.7256141763995, 'Month': '2013-12'},
 {'average_monthly_sales_prices': 739.6302006335796, 'Month': '2014-01'},
 {'average_monthly_sales_prices': 739.698

- Would like to know the yearly expenditures with suppliers (per supplier name)?   

In [180]:
query_1 = {
    '$group': {
        '_id' : {
            'year': {'$substr': ['$TransactionDate', 0, 4]},
            'supplier': '$SupplierID'
        },
        'count' : {'$sum' : '$TransactionAmount'}
    }
}

query_2 = {
    "$lookup":
    {
       "from": "Purchasing_Suppliers",
       "localField": "_id.supplier",
       "foreignField": "SupplierID",
       "as": "supplier"
     }
}

query_3 = {
    '$project': {
        '_id' : 0,
        'year': '$_id.year',
        'supplier_name' : '$supplier.SupplierName',
        'yearly_expenditures' : '$count',                
    }
}

query_4 = {
    '$sort': {'year': 1}
}

pipeline = [query_1, query_2, query_3, query_4]

r = db.Purchasing_SupplierTransactions.aggregate(pipeline)

result = list(r)

result

[{'year': '2013',
  'supplier_name': ['Fabrikam, Inc.'],
  'yearly_expenditures': 485380.49999999977},
 {'year': '2013',
  'supplier_name': ['The Phone Company'],
  'yearly_expenditures': 0.0},
 {'year': '2013',
  'supplier_name': ['Litware, Inc.'],
  'yearly_expenditures': 119343.31999999998},
 {'year': '2013',
  'supplier_name': ['Contoso, Ltd.'],
  'yearly_expenditures': 0.0},
 {'year': '2013',
  'supplier_name': ['Graphic Design Institute'],
  'yearly_expenditures': 0.0},
 {'year': '2013',
  'supplier_name': ['Northwind Electric Cars'],
  'yearly_expenditures': -3.637978807091713e-12},
 {'year': '2014',
  'supplier_name': ['Fabrikam, Inc.'],
  'yearly_expenditures': 1510955.0999999973},
 {'year': '2014',
  'supplier_name': ['Litware, Inc.'],
  'yearly_expenditures': 686937.2100000003},
 {'year': '2015',
  'supplier_name': ['Litware, Inc.'],
  'yearly_expenditures': 1353734.6900000004},
 {'year': '2015',
  'supplier_name': ['Fabrikam, Inc.'],
  'yearly_expenditures': 2837528.3999999

The marketing team:  
- Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who's customers brought the most money)?

In [200]:
# Showing the sales person that generated the most value in terms of money spent by his customers
query_1 = {
    '$match': {
        "InvoiceDate": {'$regex': '.*2013.*'}
    }
}
query_2 = {
    '$unwind' : "$InvoiceLines"
}

query_3 = {
    '$group': {
        '_id' : '$SalespersonPersonID',
        'customer_money_brought': { '$sum': { '$multiply': [ "$InvoiceLines.Quantity", "$InvoiceLines.UnitPrice" ] } },
        'invoices_amount': { '$sum': 1 }
    }
}

query_4 = {
    "$lookup":
    {
       "from": "Application_People",
       "localField": "_id",
       "foreignField": "PersonID",
       "as": "person"
     }
}

query_5 = {
    '$project': {
        '_id' : 0,
        'name': '$person.FullName',
        'customer_money_brought' : '$customer_money_brought',
        'invoices_amount' : '$invoices_amount',                
    }
}

query_6 = {
    '$sort': {'customer_money_brought': 1}
}

query_7 = {
    '$limit': 1
}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7]

r = db.Sales_Invoices.aggregate(pipeline)

result = list(r)

result

[{'name': ['Taj Shand'],
  'customer_money_brought': 2113387.3,
  'invoices_amount': 2942}]